In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [4]:
df=pd.read_excel('.\\inputs_clean.xlsx')
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels=[ list(eval(row['category_abbr'])) for index,row in df.iterrows()]
binary_matrix= mlb.fit_transform(labels)
encoded_labels = mlb.fit_transform(labels).tolist()
df['target_list']=encoded_labels
df_final = df[['clean_text','target_list']].copy()

In [5]:
label_counts = {label: sum(row) for label, row in zip(mlb.classes_, binary_matrix.T)}
l=mlb.classes_.tolist()

In [9]:
total_labels = binary_matrix.sum()
label_counts = binary_matrix.sum(axis=0)
label_percentages = (label_counts / total_labels) * 100
label_details = list(zip(mlb.classes_, label_counts, label_percentages))
label_details.sort(key=lambda x: x[1], reverse=True)
for label, count, percentage in label_details:
    print(f"标签：{label}, 数量：{count}, 百分比：{percentage:.2f}%")

标签：CTOL, 数量：8897, 百分比：23.81%
标签：RE, 数量：4934, 百分比：13.20%
标签：ARC, 数量：3883, 百分比：10.39%
标签：LOC-I, 数量：3857, 百分比：10.32%
标签：SCF-PP, 数量：3602, 百分比：9.64%
标签：LOC-G, 数量：3464, 百分比：9.27%
标签：FUEL, 数量：1251, 百分比：3.35%
标签：OTHERS, 数量：1129, 百分比：3.02%
标签：SCF-NP, 数量：1108, 百分比：2.96%
标签：CFIT, 数量：633, 百分比：1.69%
标签：UIMC, 数量：466, 百分比：1.25%
标签：GCOL, 数量：454, 百分比：1.21%
标签：RAMP, 数量：442, 百分比：1.18%
标签：LALT, 数量：408, 百分比：1.09%
标签：USOS, 数量：407, 百分比：1.09%
标签：F-POST, 数量：281, 百分比：0.75%
标签：AMAN, 数量：264, 百分比：0.71%
标签：MAC, 数量：258, 百分比：0.69%
标签：TURB, 数量：248, 百分比：0.66%
标签：F-NI, 数量：237, 百分比：0.63%
标签：LOLI, 数量：194, 百分比：0.52%
标签：EVAC, 数量：146, 百分比：0.39%
标签：WSTRW, 数量：146, 百分比：0.39%
标签：BIRD, 数量：129, 百分比：0.35%
标签：CABIN, 数量：110, 百分比：0.29%
标签：EXTL, 数量：88, 百分比：0.24%
标签：ATM, 数量：68, 百分比：0.18%
标签：ICE, 数量：68, 百分比：0.18%
标签：WILD, 数量：55, 百分比：0.15%
标签：RI, 数量：49, 百分比：0.13%
标签：GTOW, 数量：38, 百分比：0.10%
标签：ADRM, 数量：34, 百分比：0.09%
标签：SEC, 数量：13, 百分比：0.03%
标签：UNK, 数量：9, 百分比：0.02%


In [10]:
def divide_list(L):
    n = len(L)
    part_size = n // 6
    remainder = n % 6
    parts = []
    start = 0

    for i in range(6):
        end = start + part_size + (1 if i < remainder else 0)
        parts.append(L[start:end])
        start = end

    return parts

In [11]:
label_aug=[i[0] for i in label_details]
l=divide_list(label_aug)

In [12]:
no_need=l[0]
need_3=l[1]
need_5=l[2]
need_7=l[3]
need_10=l[4]
need_15=l[5]


In [13]:
def get_splite(df,lst):
    judge=[]
    for index,row in df.iterrows():
        t=eval(row['factor_list'])
        for i in lst:
            if i in t:
                judge.append(1)
                break
            if i==lst[-1]:
                judge.append(0)
    df['judge']=judge
    df_f=df[df['judge']==1]
    df_ff= df_f[['clean_text','target_list']].copy()
    return  df_ff



In [14]:
df_need3=get_splite(df,need_3)
df_need5=get_splite(df,need_5)
df_need7=get_splite(df,need_7)
df_need10=get_splite(df,need_10)
df_need15=get_splite(df,need_15)

In [15]:
df_no_need=get_splite(df,no_need)
df_no_need

,clean_text,target_list
0,the pilot wife stated that the airplane engine...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
1,the pilot attempted a simulated forced landing...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,the pilot stated that she had executed four fu...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,the pilot reported that during the takeoff rol...,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
5,the two commercial pilots one of whom was a fl...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
13573,the pilot stated that while taxiing the airpla...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
13574,the student pilot flared early and bounced the...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13577,the pilot was on a cross-country flight and no...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13578,on the day of the accident the pilot reported ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [16]:
import random
import nltk
from nltk.corpus import wordnet
from itertools import chain

nltk.download('wordnet')

def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()

    for _ in range(n):
        word_to_replace = random.choice(words)
        synonyms = wordnet.synsets(word_to_replace)

        if synonyms:
            synonym_words = set(chain.from_iterable([syn.lemma_names() for syn in synonyms]))
            synonym_words.discard(word_to_replace)  # 删除原词
            if synonym_words:
                synonym_word = random.choice(list(synonym_words))
                new_words[new_words.index(word_to_replace)] = synonym_word

    return ' '.join(new_words)


def random_insertion(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        word_to_insert = random.choice(words)
        synonyms = wordnet.synsets(word_to_insert)

        if synonyms:
            synonym_words = set(chain.from_iterable([syn.lemma_names() for syn in synonyms]))
            if synonym_words:
                new_word = random.choice(list(synonym_words))
                insert_position = random.randint(0, len(words))
                words.insert(insert_position, new_word)

    return ' '.join(words)


def random_deletion(sentence, p=0.5):
    words = sentence.split()
    if len(words) == 1: 
        return sentence
    remaining_words = [word for word in words if random.random() > p]
    return ' '.join(remaining_words) if remaining_words else random.choice(words)


def random_swap(sentence, n=1):
    words = sentence.split()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]

    return ' '.join(words)

def augment_text(sentence, n_synonyms=1, n_insertions=1, p_deletion=0.5, n_swaps=1):
    sentence = synonym_replacement(sentence, n=n_synonyms)
    sentence = random_insertion(sentence, n=n_insertions)
    sentence = random_deletion(sentence, p=p_deletion)
    sentence = random_swap(sentence, n=n_swaps)
    return sentence


[nltk_data] Downloading package wordnet to C:\Users\Rick
[nltk_data]     Qin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
def augment_dataframe(df, text_column='clean_text', target_column='target_list', augment_times=1):
    augmented_texts = []
    target_labels = []
    
    for _, row in df.iterrows():
        text = row[text_column]
        label = row[target_column]
        
        
        augmented_texts.append(text)
        target_labels.append(label)
        
        
        for _ in range(augment_times):
            augmented_text = augment_text(text)
            augmented_texts.append(augmented_text)
            target_labels.append(label)
    

    augmented_df = pd.DataFrame({
        text_column: augmented_texts,
        target_column: target_labels
    })
    
    return augmented_df

In [18]:
df_augmented_need3 = augment_dataframe(df_need3, text_column='clean_text', target_column='target_list', augment_times=2)
df_augmented_need5=augment_dataframe(df_need5, text_column='clean_text', target_column='target_list', augment_times=4)
df_augmented_need7=augment_dataframe(df_need7, text_column='clean_text', target_column='target_list', augment_times=6)
df_augmented_need10=augment_dataframe(df_need10, text_column='clean_text', target_column='target_list', augment_times=9)
df_augmented_need15=augment_dataframe(df_need15, text_column='clean_text', target_column='target_list', augment_times=20)

In [19]:
df_concatenated = pd.concat([df_no_need,df_augmented_need3,df_augmented_need5,df_augmented_need7,df_augmented_need10,df_augmented_need15], axis=0, ignore_index=True)

In [20]:
df_concatenated.to_excel('inputs_augment_new_factor.xlsx',index=False)